In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER_IN = 161
VER_OUT = 161
PART = 3

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
users = pd.read_parquet(f'../../data/infer_with_features/train_v{VER_IN}_LB_{PART}.pqt')

In [4]:
print( users.shape )
users.head()

(41795000, 94)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0
3,12899779,731692,3,5.990,1.0,-3.0,-3.0,-3.0,0.5,-3.0,-3.000000,1.5,0.300000,4.0,10.675,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,1.0,0.140000,50.0,28.0,1.785714,0.404566,NaN,NaN,-1.000000,0.25,0.500000,-1.0,-1.0,-1.000000,0.0,-1.0,4.0,2.0,-1.0,-1.0,-1.0,1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,1.527525,1.527525,0.162791,86.0,41.0,52135.464844,295261.03125,2.097561,0.430061,17917.339844,174344.12500,0.285714,7.0,4.0,53310.429688,485310.43750,1.750000,0.487950,11891.376953,79692.445312,3.0,4.0,0.0,2.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0
4,12899779,1246235,4,5.935,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.050000,-3.0,0.900000,3.0,4.250,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.038974,975.0,664.0,1.468374,0.208935,NaN,NaN,0.235294,0.00,0.000000,0.1,0.0,0.083333,0.0,17.0,8.0,10.0,10.0,4.0,12.0,5.0,12.0,8.0,10.0,4.0,4.0,10.0,4.0,4.391551,4.391551,0.053419,1872.0,1228.0,52150.824219,312135.43750,1.524430,0.240987,18081.478516,162183.68750,0.090909,66.0,52.0,47462.273438,338080.46875,1.269231,0.381385,19521.773438,170804.906250,37.0,36.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,79200.0,338

In [5]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')
#df['type'] = df['type'].map({'clicks':0,'carts':1,'orders':2}).astype('int8')

print( df.shape )
df.head()

(6928123, 4)


,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


In [6]:
df = df.sort_values(['session','ts']).reset_index(drop=True)

In [7]:
tmp = df.groupby('session').aid.agg('last').rename('last')

In [8]:
%%time
users = users.merge(tmp.to_pandas(), left_on='user', right_index=True, how='left')

CPU times: user 4.65 s, sys: 2.21 s, total: 6.86 s
Wall time: 6.51 s


In [9]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,last
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,59625
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,59625
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,59625
3,12899779,731692,3,5.990,1.0,-3.0,-3.0,-3.0,0.5,-3.0,-3.000000,1.5,0.300000,4.0,10.675,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,1.0,0.140000,50.0,28.0,1.785714,0.404566,NaN,NaN,-1.000000,0.25,0.500000,-1.0,-1.0,-1.000000,0.0,-1.0,4.0,2.0,-1.0,-1.0,-1.0,1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,1.527525,1.527525,0.162791,86.0,41.0,52135.464844,295261.03125,2.097561,0.430061,17917.339844,174344.12500,0.285714,7.0,4.0,53310.429688,485310.43750,1.750000,0.487950,11891.376953,79692.445312,3.0,4.0,0.0,2.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,59625
4,12899779,1246235,4,5.935,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.050000,-3.0,0.900000,3.0,4.250,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.038974,975.0,664.0,1.468374,0.208935,NaN,NaN,0.235294,0.00,0.000000,0.1,0.0,0.083333,0.0,17.0,8.0,10.0,10.0,4.0,12.0,5.0,12.0,8.0,10.0,4.0,4.0,10.0,4.0,4.391551,4.391551,0.053419,1872.0,1228.0,52150.824219,312135.43750,1.524430,0.240987,18081.478516,162183.68750,0.090909,66.0,52.0,47462.273438,338080.46875,1.269231,0.381385,19521.773438,170804.906250,37.0,36.0,2.0,2.0,1.

In [10]:
%%time
x = users.item.values
y = users['last'].values

CPU times: user 283 µs, sys: 86 µs, total: 369 µs
Wall time: 293 µs


In [11]:
users = users.drop(['last'],axis=1)

In [12]:
embed = np.load('../../data/item_user_features/item_embed_32_LB.npy')
z = np.sqrt( np.sum(embed * embed,axis=1) )
embed = embed/ np.reshape(z,(-1,1))
embed.shape
#(1855603, 32)

(1855603, 32)

In [13]:
%%time
e1 = embed[x,]
e2 = embed[y,]

CPU times: user 3.19 s, sys: 992 ms, total: 4.18 s
Wall time: 4.18 s


In [14]:
e1.shape

(41795000, 32)

In [15]:
%%time
z = np.sum( e1*e2,axis=1 )

CPU times: user 2.58 s, sys: 597 ms, total: 3.18 s
Wall time: 3.18 s


In [16]:
users['z1'] = z

In [17]:
users.z1.dtype

dtype('float32')

In [18]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954612
3,12899779,731692,3,5.990,1.0,-3.0,-3.0,-3.0,0.5,-3.0,-3.000000,1.5,0.300000,4.0,10.675,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,1.0,0.140000,50.0,28.0,1.785714,0.404566,NaN,NaN,-1.000000,0.25,0.500000,-1.0,-1.0,-1.000000,0.0,-1.0,4.0,2.0,-1.0,-1.0,-1.0,1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,1.527525,1.527525,0.162791,86.0,41.0,52135.464844,295261.03125,2.097561,0.430061,17917.339844,174344.12500,0.285714,7.0,4.0,53310.429688,485310.43750,1.750000,0.487950,11891.376953,79692.445312,3.0,4.0,0.0,2.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.956417
4,12899779,1246235,4,5.935,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.050000,-3.0,0.900000,3.0,4.250,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.038974,975.0,664.0,1.468374,0.208935,NaN,NaN,0.235294,0.00,0.000000,0.1,0.0,0.083333,0.0,17.0,8.0,10.0,10.0,4.0,12.0,5.0,12.0,8.0,10.0,4.0,4.0,10.0,4.0,4.391551,4.391551,0.053419,1872.0,1228.0,52150.824219,312135.43750,1.524430,0.240987,18081.478516,162183.68750,0.090909,66.0,52.0,47462.273438,338080.46875,1.269231,0.381385,19521.773438,170804.906250,37.0,36.0,

In [19]:
users2 = cudf.DataFrame( users[['user','item']] )
print( users2.shape )

(41795000, 2)


In [20]:
for k in range(5):
    tmp = df.loc[df['type']!=0].reset_index(drop=True) 
    tmp['n'] = tmp.groupby('session').aid.cumcount()
    tmp = tmp.loc[tmp.n==k]
    tmp = tmp[['session','aid']]
    tmp.columns = ['user','last_buy']

    users2 = users2.merge(tmp, on='user', how='left')

    x = users2.loc[users2.last_buy.notnull()].item.to_numpy()
    y = users2.loc[users2.last_buy.notnull()]['last_buy'].to_numpy().astype('int32')

    e1 = embed[x,]
    e2 = embed[y,]

    z = np.sum( e1*e2,axis=1 )

    users2[f'z{k+2}'] = -1
    users2.loc[users2.last_buy.notnull(),f'z{k+2}'] = z

    users2 = users2.drop(['last_buy'],axis=1)
    
    print(k,users2.shape)

0 (41795000, 3)
1 (41795000, 4)
2 (41795000, 5)
3 (41795000, 6)
4 (41795000, 7)


In [21]:
del df, tmp, e1, e2, z, x, y, embed
gc.collect()

458

In [22]:
print( users2.shape )
users2.head()

(41795000, 7)


,user,item,z2,z3,z4,z5,z6
0,12899814,676888,-1.0,-1.0,-1.0,-1.0,-1.0
1,12899814,755147,-1.0,-1.0,-1.0,-1.0,-1.0
2,12899814,437279,-1.0,-1.0,-1.0,-1.0,-1.0
3,12899814,968314,-1.0,-1.0,-1.0,-1.0,-1.0
4,12899814,351587,-1.0,-1.0,-1.0,-1.0,-1.0


In [23]:
print( users.shape)
users.head()

(41795000, 95)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954612
3,12899779,731692,3,5.990,1.0,-3.0,-3.0,-3.0,0.5,-3.0,-3.000000,1.5,0.300000,4.0,10.675,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,1.0,0.140000,50.0,28.0,1.785714,0.404566,NaN,NaN,-1.000000,0.25,0.500000,-1.0,-1.0,-1.000000,0.0,-1.0,4.0,2.0,-1.0,-1.0,-1.0,1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,1.527525,1.527525,0.162791,86.0,41.0,52135.464844,295261.03125,2.097561,0.430061,17917.339844,174344.12500,0.285714,7.0,4.0,53310.429688,485310.43750,1.750000,0.487950,11891.376953,79692.445312,3.0,4.0,0.0,2.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.956417
4,12899779,1246235,4,5.935,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.050000,-3.0,0.900000,3.0,4.250,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.038974,975.0,664.0,1.468374,0.208935,NaN,NaN,0.235294,0.00,0.000000,0.1,0.0,0.083333,0.0,17.0,8.0,10.0,10.0,4.0,12.0,5.0,12.0,8.0,10.0,4.0,4.0,10.0,4.0,4.391551,4.391551,0.053419,1872.0,1228.0,52150.824219,312135.43750,1.524430,0.240987,18081.478516,162183.68750,0.090909,66.0,52.0,47462.273438,338080.46875,1.269231,0.381385,19521.773438,170804.906250,37.0,36.0,

In [24]:
%%time
users = users.merge(users2.drop_duplicates(['user','item']).to_pandas(), on=['user','item'], how='left')

CPU times: user 29.6 s, sys: 8.16 s, total: 37.8 s
Wall time: 37.8 s


In [25]:
print( users.shape )
users.head()

(41795000, 100)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954612,-1.0,-1.0,-1.0,-1.0,-1.0
3,12899779,731692,3,5.990,1.0,-3.0,-3.0,-3.0,0.5,-3.0,-3.000000,1.5,0.300000,4.0,10.675,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,1.0,0.140000,50.0,28.0,1.785714,0.404566,NaN,NaN,-1.000000,0.25,0.500000,-1.0,-1.0,-1.000000,0.0,-1.0,4.0,2.0,-1.0,-1.0,-1.0,1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,1.527525,1.527525,0.162791,86.0,41.0,52135.464844,295261.03125,2.097561,0.430061,17917.339844,174344.12500,0.285714,7.0,4.0,53310.429688,485310.43750,1.750000,0.487950,11891.376953,79692.445312,3.0,4.0,0.0,2.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.956417,-1.0,-1.0,-1.0,-1.0,-1.0
4,12899779,1246235,4,5.935,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.050000,-3.0,0.900000,3.0,4.250,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.038974,975.0,664.0,1.468374,0.208935,NaN,NaN,0.235294,0.00,0.000000,0.1,0.0,0.083333,0.0,17.0,8.0,10.0,10.0,4.0,12.0,5.0,12.0,8.0,10.0,4.0,4.0,10.0,4.0,4.391551,4.391551,0.053419,1872.0,1228.0,52150.824219,312135.43750,1.524430,0.240987,18081.47851

In [26]:
for c in ['z1']+[f'z{x+2}' for x in range(5)]:
    print(c)
    users[c] = users[c].astype('float32')

z1
z2
z3
z4
z5
z6


# Write Parquet

In [27]:
for c in users.columns:
    print(c, users[c].dtype)
    if str(users[c].dtype) in ['Float32','Int32','float64','int64']:
        print('###',c)
        users[c] = users[c].astype('float32')

user int32
item int32
pos int32
wgt float32
score_1 float32
score_3 float32
score_4 float32
score_5 float32
score_6 float32
score_7 float32
score_10 float32
score_11 float32
score_12 float32
score_13 float32
score_30 float32
score_14 float32
score_15 float32
score_16 float32
score_17 float32
score_18 float32
score_19 float32
score_20 float32
score_21 float32
score_22 float32
score_23 float32
score_24 float32
score_25 float32
score_26 float32
buy_ratio_5 float32
count_item_5 float32
count_user_5 float32
repeat_5 float32
buy_ratio_std_5 float32
item_day_std_5 float32
user_day_std_5 float32
buy_ratio_0_6 float32
buy_ratio_1_6 float32
buy_ratio_2_6 float32
buy_ratio_3_6 float32
buy_ratio_4_6 float32
buy_ratio_5_6 float32
buy_ratio_6_6 float32
count_item_0_6 float32
count_item_1_6 float32
count_item_2_6 float32
count_item_3_6 float32
count_item_4_6 float32
count_item_5_6 float32
count_item_6_6 float32
count_user_0_6 float32
count_user_1_6 float32
count_user_2_6 float32
count_user_3_6 float3

In [28]:
%%time
users.to_parquet(f'../../data/infer_with_features/train_v{VER_OUT}_LB_{PART}.pqt',index=False)

CPU times: user 2min, sys: 4.21 s, total: 2min 4s
Wall time: 1min 36s
